In [71]:
# Зависимости
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import os

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVR, SVC
from sklearn.tree import DecisionTreeRegressor
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor 

from sklearn.linear_model import LinearRegression,Lasso, Ridge, ElasticNet, LogisticRegression
from sklearn.metrics import mean_squared_error

In [72]:
# Инициализируем все известные генераторы случаынйх чисел / Setting all known random seeds
my_code = "Севастьянов"
seed_limit = 2 ** 32
my_seed = int.from_bytes(my_code.encode(), "little") % seed_limit

os.environ['PYTHONHASHSEED']=str(my_seed)

random.seed(my_seed)

np.random.seed(my_seed)

In [73]:
# Читаем данные из файла
train_data = pd.read_csv("datasets/rus_stocks_funds_train.csv")

In [74]:
train_data.head()

,Total issued ordinary shares,Number of shares available for sale,Number of employees,Number of shareholders,Price/Profit,Debt/Assets
0,7.210000e+09,2.870000e+09,110809.6875,3190.000000,9.870000,0.27
1,1.060000e+10,1.130000e+09,318000.0000,28230.000000,13.910000,0.30
2,2.249000e+10,1.027000e+10,329570.0000,61044.948718,4.080000,0.25
3,4.995200e+08,1.964200e+08,59380.0000,81.000000,2.910000,1.31
4,9.520000e+09,1.309960e+11,110809.6875,61044.948718,15.525191,0.35


In [75]:
classes = train_data['species']
plt.hist(classes)
plt.show()

KeyError: 'species'

In [ ]:
# Определим размер валидационной выборки
val_size = round(0.2*len(train_data))
print(val_size)

In [ ]:
# Создадим обучающую и валидационную выборки
random_state = my_seed
train, val = train_test_split(train_data, test_size=val_size, random_state=random_state)
print(len(train), len(val))

In [76]:
# Значения в числовых столбцах преобразуем к отрезку [0,1].
# Для настройки скалировщика используем только обучающую выборку.
y_column = 'Debt/Assets'
x_columns = [e for e in train_data.columns if e != y_column]

ct = ColumnTransformer(transformers=[('numerical', MinMaxScaler(), x_columns)], remainder='passthrough')

ct.fit(train)

ColumnTransformer(remainder='passthrough',
                  transformers=[('numerical', MinMaxScaler(),
                                 ['Total issued ordinary shares',
                                  'Number of shares available for sale',
                                  'Number of employees',
                                  'Number of shareholders', 'Price/Profit'])])

In [77]:
# Преобразуем значения, тип данных приводим к DataFrame
sc_train = pd.DataFrame(ct.transform(train))
sc_val = pd.DataFrame(ct.transform(val))

In [78]:
# Устанавливаем названия столбцов
column_names = list(x_columns) + [y_column]
sc_train.columns = column_names
sc_val.columns = column_names

In [79]:
sc_train

,Total issued ordinary shares,Number of shares available for sale,Number of employees,Number of shareholders,Price/Profit,Debt/Assets
0,0.000671,0.025882,0.978632,0.111928,0.077764,0.18
1,0.000066,0.024067,0.233354,0.098469,0.033986,0.06
2,0.377806,0.898701,0.203761,0.149645,0.019101,0.13
3,0.000065,0.024049,0.000000,0.000027,0.027048,0.29
4,0.001044,0.030674,0.233354,0.076527,0.002547,0.01
...,...,...,...,...,...,...
258,0.000310,0.024024,0.676068,0.069197,0.056029,0.30
259,0.000066,0.049084,0.233354,0.149645,0.059366,0.84
260,0.000066,0.049084,0.233354,0.149645,0.043163,0.04
261,0.036859,0.069533,0.233354,1.000000,0.063121,0.28


In [80]:
# Отберем необходимые параметры
x_train = sc_train[x_columns]
x_val = sc_val[x_columns]

y_train = (sc_train[y_column].values).flatten()
y_val = (sc_val[y_column].values).flatten()

In [81]:
model = DecisionTreeRegressor(criterion='absolute_error')

In [82]:
# Обучим модель
for model in r_models:
    print(model)
    model.fit(x_train, y_train)

ElasticNet(alpha=0.5)
ElasticNet(alpha=0.5, l1_ratio=0.75)
LinearRegression()
Lasso()
DecisionTreeRegressor()
SVR()
Ridge()
KNeighborsRegressor()
KNeighborsRegressor(n_neighbors=10)
KNeighborsRegressor(n_neighbors=15)
SVR(kernel='linear')
SVR(kernel='poly')
SVR()
SVR(kernel='sigmoid')
DecisionTreeRegressor()
DecisionTreeRegressor(criterion='friedman_mse')
DecisionTreeRegressor(criterion='absolute_error')


In [83]:
# Проверим работу обученной модели на валидационной выборке
mses = []
for model in r_models:
    val_pred = model.predict(x_val)
    mse = mean_squared_error(y_val, val_pred)
    mses.append(mse)
    print(model, '\t', mse)

ElasticNet(alpha=0.5) 	 0.08619088896691851
ElasticNet(alpha=0.5, l1_ratio=0.75) 	 0.08619088896691851
LinearRegression() 	 0.07951659284515666
Lasso() 	 0.08619088896691851
DecisionTreeRegressor() 	 0.05266363636363637
SVR() 	 0.07243854705092079
Ridge() 	 0.07944872949925155
KNeighborsRegressor() 	 0.0962947272727273
KNeighborsRegressor(n_neighbors=10) 	 0.09550058314398581
KNeighborsRegressor(n_neighbors=15) 	 0.08181331469452084
SVR(kernel='linear') 	 0.08232611789151022
SVR(kernel='poly') 	 0.07969140025311572
SVR() 	 0.07243854705092079
SVR(kernel='sigmoid') 	 66.31079644691292
DecisionTreeRegressor() 	 0.05493636363636364
DecisionTreeRegressor(criterion='friedman_mse') 	 0.06170757575757576
DecisionTreeRegressor(criterion='absolute_error') 	 0.0475


In [84]:
i_min = mses.index(min(mses))
best_r_model = r_models[i_min]
best_r_model

DecisionTreeRegressor(criterion='absolute_error')

In [85]:
val_pred

array([0.21, 0.19, 0.01, 0.76, 0.48, 0.41, 0.24, 0.31, 0.24, 0.25, 0.01,
       0.25, 0.31, 0.35, 0.51, 0.29, 0.01, 0.29, 0.15, 0.11, 0.54, 0.02,
       0.53, 0.18, 0.19, 0.11, 0.07, 0.12, 0.25, 0.15, 0.37, 0.84, 0.12,
       0.18, 0.12, 0.13, 0.08, 0.01, 0.6 , 0.22, 0.54, 0.12, 0.52, 0.15,
       0.35, 0.35, 0.54, 0.07, 0.6 , 0.27, 0.25, 0.25, 0.54, 0.12, 0.17,
       0.24, 0.5 , 0.13, 0.63, 0.2 , 0.18, 0.47, 0.54, 0.21, 0.07, 0.02])

In [86]:
test = pd.read_csv("datasets/rus_stocks_funds_test.csv")

In [87]:
test[y_column] = 0.0

In [88]:
test

,Total issued ordinary shares,Number of shares available for sale,Number of employees,Number of shareholders,Price/Profit,Debt/Assets
0,3.644700e+11,1.017600e+11,5840.0000,306350.000000,7.860000,0.0
1,6.968900e+08,4.339700e+08,110809.6875,39210.000000,5.670000,0.0
2,1.580000e+07,3.880000e+06,110809.6875,29.000000,7.060000,0.0
3,1.295000e+08,3.573000e+07,110809.6875,25.000000,14.940000,0.0
4,2.400000e+09,1.309960e+11,110809.6875,61044.948718,15.525191,0.0
...,...,...,...,...,...,...
77,1.528600e+08,1.309960e+11,110809.6875,61044.948718,6.270000,0.0
78,1.295000e+08,2.634000e+07,110809.6875,25.000000,15.525191,0.0
79,2.246000e+10,0.000000e+00,281550.0000,61044.948718,6.340000,0.0
80,1.295000e+08,3.371000e+07,110809.6875,25.000000,6.320000,0.0


In [89]:
sc_test = pd.DataFrame(ct.transform(test))
sc_test.columns = column_names

In [90]:
x_test = sc_test[x_columns]

In [91]:
test[y_column] = model.predict(x_test)

In [92]:
test.head()

,Total issued ordinary shares,Number of shares available for sale,Number of employees,Number of shareholders,Price/Profit,Debt/Assets
0,3.644700e+11,1.017600e+11,5840.0000,306350.000000,7.860000,0.18
1,6.968900e+08,4.339700e+08,110809.6875,39210.000000,5.670000,0.12
2,1.580000e+07,3.880000e+06,110809.6875,29.000000,7.060000,0.27
3,1.295000e+08,3.573000e+07,110809.6875,25.000000,14.940000,0.48
4,2.400000e+09,1.309960e+11,110809.6875,61044.948718,15.525191,0.76


In [93]:
test.to_csv('task5_results/Севастьянов Н.Г.csv', index=False)